In [1]:
# -----------------------------------------------------------------------------
# NOTEBOOK 1: Installation and Basic Text Generation
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

printmd("# 1. Basic Text Generation - Understanding Autoregressive Behavior 🚀")
printmd("This is the core function of a decoder-only model: predicting the next token in a sequence.")
printmd("---")

printmd("## 📥 Installation (Run this cell once)")
printmd("```bash\n!pip install transformers torch\n```")

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# Set model configuration for cleaner generation output
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Input text
prompt = "The future of artificial intelligence is"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

print(f"Input: {prompt}")
print(f"Input tokens shape: {input_ids.shape}")

# Generate text
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True
    )

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"\nGenerated Text:\n{generated_text}")

printmd("## Key Insight")
printmd("GPT generates text **one token at a time**, using previously generated tokens as context. This is **autoregressive generation**—the hallmark of decoder-only models.")

/Users/nitin.sharma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nitin.sharma/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Basic Text Generation - Understanding Autoregressive Behavior 🚀

This is the core function of a decoder-only model: predicting the next token in a sequence.

---

## 📥 Installation (Run this cell once)

```bash
!pip install transformers torch
```

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input: The future of artificial intelligence is
Input tokens shape: torch.Size([1, 6])

Generated Text:
The future of artificial intelligence is not in the hands of corporations, but in the hands of people, who are capable of creating the best possible world. This is something that we are not just talking about, but a fundamental change that is happening in our


## Key Insight

GPT generates text **one token at a time**, using previously generated tokens as context. This is **autoregressive generation**—the hallmark of decoder-only models.

In [2]:
# -----------------------------------------------------------------------------
# NOTEBOOK 2: Step-by-Step Token Prediction
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn.functional as F
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded (assuming Notebook 1 was run or run these lines)
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

printmd("# 2. Step-by-Step Token Prediction - Visualizing the Decoder Process 🧪")
printmd("We manually step through the generation loop to see the model's top predictions at each step.")
printmd("---")

prompt = "Machine learning is"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
print(f"Starting prompt: {prompt}\n")
print("=" * 60)

current_ids = input_ids.clone()
num_tokens_to_generate = 5
for i in range(num_tokens_to_generate):
    with torch.no_grad():
        outputs = model(current_ids)
        next_token_logits = outputs.logits[0, -1, :]

    probs = F.softmax(next_token_logits, dim=-1)
    top_probs, top_indices = torch.topk(probs, 5)

    print(f"\nStep {i+1}:")
    print(f"Current text: '{tokenizer.decode(current_ids[0])}'")
    print(f"Top 5 predictions for next token:")

    for prob, idx in zip(top_probs, top_indices):
        token = tokenizer.decode([idx]).replace('\n', '\\n').replace(' ', '·')
        print(f"  '{token}' -> {prob.item():.4f}")

    # Use greedy decoding for a clear, consistent demonstration
    next_token_id = torch.argmax(next_token_logits).unsqueeze(0).unsqueeze(0)
    current_ids = torch.cat([current_ids, next_token_id], dim=-1)

    selected_token = tokenizer.decode(next_token_id[0]).replace('\n', '\\n').replace(' ', '·')
    print(f"Selected (Greedy): '{selected_token}'")

print("\n" + "=" * 60)
print(f"Final generated text: {tokenizer.decode(current_ids[0])}")

printmd("## Key Insight")
printmd("At each step, GPT looks at **ALL previous tokens** and predicts the next one. This sequential dependence confirms its decoder-only, autoregressive nature.")

# 2. Step-by-Step Token Prediction - Visualizing the Decoder Process 🧪

We manually step through the generation loop to see the model's top predictions at each step.

---

Starting prompt: Machine learning is


Step 1:
Current text: 'Machine learning is'
Top 5 predictions for next token:
  '·a' -> 0.1383
  '·the' -> 0.0552
  '·not' -> 0.0419
  '·an' -> 0.0360
  '·one' -> 0.0174
Selected (Greedy): '·a'

Step 2:
Current text: 'Machine learning is a'
Top 5 predictions for next token:
  '·very' -> 0.0512
  '·great' -> 0.0395
  '·big' -> 0.0238
  '·new' -> 0.0225
  '·powerful' -> 0.0217
Selected (Greedy): '·very'

Step 3:
Current text: 'Machine learning is a very'
Top 5 predictions for next token:
  '·powerful' -> 0.0764
  '·important' -> 0.0678
  '·complex' -> 0.0664
  '·good' -> 0.0419
  '·interesting' -> 0.0361
Selected (Greedy): '·powerful'

Step 4:
Current text: 'Machine learning is a very powerful'
Top 5 predictions for next token:
  '·tool' -> 0.2763
  '·and' -> 0.0863
  '·way' -> 0.0759
  '·technology' -> 0.0526
  ',' -> 0.0329
Selected (Greedy): '·tool'

Step 5:
Current text: 'Machine learning is a very powerful tool'
Top 5 predictions for next token

## Key Insight

At each step, GPT looks at **ALL previous tokens** and predicts the next one. This sequential dependence confirms its decoder-only, autoregressive nature.

In [3]:
# -----------------------------------------------------------------------------
# NOTEBOOK 3: Understanding Masked Self-Attention
# -----------------------------------------------------------------------------

import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

printmd("# 3. Understanding Masked Self-Attention (The Causal Mask) 🎭")
printmd("The masked attention mechanism is what defines the GPT decoder's inability to 'look ahead' at future tokens.")
printmd("---")

sequence_length = 5
# Create attention mask for decoder (causal/autoregressive mask)
# This prevents positions from attending to future positions
attention_mask = torch.tril(torch.ones(sequence_length, sequence_length))

printmd("### Causal Attention Mask (Decoder-Only):")
printmd("`1 = can attend, 0 = cannot attend (masked)`")
print(attention_mask)

print("\nExplanation:")
print("- Row i (Query token): can only attend to columns j <= i (Key tokens)")
print("- Row 0 (token 0): can only see token 0 (itself)")
print("- Row 4 (token 4): can see tokens 0, 1, 2, 3, 4")

printmd("## Key Insight")
printmd("The **triangular mask** ensures that when predicting token N, the model can only see tokens 0 to N-1. This left-to-right flow is crucial for autoregressive generation.")

# 3. Understanding Masked Self-Attention (The Causal Mask) 🎭

The masked attention mechanism is what defines the GPT decoder's inability to 'look ahead' at future tokens.

---

### Causal Attention Mask (Decoder-Only):

`1 = can attend, 0 = cannot attend (masked)`

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

Explanation:
- Row i (Query token): can only attend to columns j <= i (Key tokens)
- Row 0 (token 0): can only see token 0 (itself)
- Row 4 (token 4): can see tokens 0, 1, 2, 3, 4


## Key Insight

The **triangular mask** ensures that when predicting token N, the model can only see tokens 0 to N-1. This left-to-right flow is crucial for autoregressive generation.

In [4]:
# -----------------------------------------------------------------------------
# NOTEBOOK 4: Extracting and Analyzing Model Outputs
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

printmd("# 4. Extracting and Analyzing Model Outputs 🔬")
printmd("We look inside the model to see the output shapes of the logits, hidden states, and attention weights.")
printmd("---")

prompt = "Python programming is"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Get detailed model outputs including hidden states and attentions
with torch.no_grad():
    outputs = model(input_ids, output_hidden_states=True, output_attentions=True)

logits = outputs.logits                  # Predictions for next token
hidden_states = outputs.hidden_states    # Hidden states from all layers
attentions = outputs.attentions          # Attention weights from all layers

print(f"Input: {prompt}")
print(f"Input shape: {input_ids.shape}")
print(f"\nModel has {len(hidden_states)} layers (including embedding)")
print(f"GPT-2 Hidden Dimension (n_embd): {model.config.n_embd}")

printmd("### Logits Shape (Output Layer)")
print(f"Logits shape: {logits.shape}")
print(f"  - Sequence length: {logits.shape[1]}")
print(f"  - Vocabulary size: {logits.shape[2]}")

printmd("\n### Hidden States Shape")
print(f"Hidden state for last layer shape: {hidden_states[-1].shape}")

printmd("\n### Attention Weights Shape")
print(f"Attention weights for layer 0 shape: {attentions[0].shape}")
print(f"  - Number of attention heads: {attentions[0].shape[1]}")

# Predict next token
next_token_logits = logits[0, -1, :]
next_token_id = torch.argmax(next_token_logits).item()
next_token = tokenizer.decode([next_token_id])
print(f"\nPredicted next token: '{next_token}'")

printmd("## Key Insight")
printmd("GPT processes the entire input sequence through multiple decoder layers, with the final layer's output (logits) being converted to vocabulary probabilities.")

# 4. Extracting and Analyzing Model Outputs 🔬

We look inside the model to see the output shapes of the logits, hidden states, and attention weights.

---

Input: Python programming is
Input shape: torch.Size([1, 3])

Model has 13 layers (including embedding)
GPT-2 Hidden Dimension (n_embd): 768


### Logits Shape (Output Layer)

Logits shape: torch.Size([1, 3, 50257])
  - Sequence length: 3
  - Vocabulary size: 50257



### Hidden States Shape

Hidden state for last layer shape: torch.Size([1, 3, 768])



### Attention Weights Shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [5]:
# -----------------------------------------------------------------------------
# NOTEBOOK 5: Comparing Different Decoding Strategies
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

printmd("# 5. Comparing Different Decoding Strategies 🧠")
printmd("Decoder-only models can generate text in various ways by altering how the next token is sampled from the probability distribution.")
printmd("---")

prompt = "Once upon a time"
print(f"Prompt: {prompt}\n")
print("=" * 80)

input_ids = tokenizer.encode(prompt, return_tensors="pt")

# 1. Greedy Decoding
printmd("\n**1. GREEDY DECODING (Deterministic):** Always picks the single highest probability token.")
output = model.generate(input_ids, max_length=30, do_sample=False)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# 2. Sampling with Temperature
printmd("\n**2. SAMPLING with temperature=0.7 (Randomness):** Introduces variance based on the probability distribution.")
output = model.generate(input_ids, max_length=30, do_sample=True, temperature=0.7)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# 3. Top-k Sampling
printmd("\n**3. TOP-K SAMPLING (k=50):** Only samples from the top 50 most probable tokens.")
output = model.generate(input_ids, max_length=30, do_sample=True, top_k=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# 4. Top-p (Nucleus) Sampling
printmd("\n**4. TOP-P SAMPLING (p=0.9):** Samples from the smallest set of tokens whose cumulative probability exceeds 0.9.")
output = model.generate(input_ids, max_length=30, do_sample=True, top_p=0.9)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# 5. Beam Search
printmd("\n**5. BEAM SEARCH (num_beams=5):** Keeps track of 5 best sequences at each step to find the globally optimal (most probable) output.")
output = model.generate(input_ids, max_length=30, num_beams=5, early_stopping=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

printmd("## Key Insight")
printmd("Decoder-only models support various decoding strategies, all based on the same principle: use the predicted token probabilities to select the next step in the sequence.")

# 5. Comparing Different Decoding Strategies 🧠

Decoder-only models can generate text in various ways by altering how the next token is sampled from the probability distribution.

---

Prompt: Once upon a time




**1. GREEDY DECODING (Deterministic):** Always picks the single highest probability token.

...


**5. BEAM SEARCH (num_beams=5):** Keeps track of 5 best sequences at each step to find the globally optimal (most probable) output.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, it was said, there would be a time when the world would be a better place.

It was a time when


## Key Insight

Decoder-only models support various decoding strategies, all based on the same principle: use the predicted token probabilities to select the next step in the sequence.

In [6]:
# -----------------------------------------------------------------------------
# NOTEBOOK 6: Understanding Context Window
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

printmd("# 6. Understanding Context Window 🪟")
printmd("The context window defines the maximum number of tokens a model can attend to at any given time.")
printmd("---")

max_length = model.config.n_positions
print(f"GPT-2 maximum context length: {max_length} tokens\n")

# Create a long prompt
long_text = "AI " * 600  # Creates a text over 1024 tokens
input_ids = tokenizer.encode(long_text, return_tensors="pt")

print(f"Long text token count: {input_ids.shape[1]} tokens")

if input_ids.shape[1] > max_length:
    print(f"Text exceeds max length! Truncating to last {max_length} tokens...")
    # Truncate the input to fit the context window
    input_ids = input_ids[:, -max_length:]
    print(f"Truncated token count: {input_ids.shape[1]} tokens")

# Generate with truncated input
with torch.no_grad():
    output = model.generate(input_ids, max_length=input_ids.shape[1] + 20)

print(f"\nGenerated: ...{tokenizer.decode(output[0][-50:], skip_special_tokens=True)}")

printmd("## Key Insight")
printmd("Decoder-only models have a **fixed context window**. They can only attend to a limited number of previous tokens. Input exceeding this limit is simply ignored or truncated.")

# 6. Understanding Context Window 🪟

The context window defines the maximum number of tokens a model can attend to at any given time.

---

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 maximum context length: 1024 tokens

Long text token count: 601 tokens

Generated: ... AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI AI  AI  AI  AI  AI  AI  AI  AI


## Key Insight

Decoder-only models have a **fixed context window**. They can only attend to a limited number of previous tokens. Input exceeding this limit is simply ignored or truncated.

In [ ]:
# -----------------------------------------------------------------------------
# NOTEBOOK 8: Using Different GPT Model Sizes
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

printmd("# 8. Using Different GPT Model Sizes 🐘")
printmd("All GPT-2 variants share the same decoder-only architecture, differing only in the number of layers and hidden size.")
printmd("---")

models_info = {
    "gpt2": "124M parameters",
    "gpt2-medium": "355M parameters",
    "gpt2-large": "774M parameters",
    "gpt2-xl": "1.5B parameters"
}
prompt = "Artificial intelligence will"
printmd("Comparing GPT-2 model sizes:")
print("=" * 80)

# Load and test the smallest model
for model_name_check in ["gpt2"]:  # Use "gpt2-medium" or "gpt2-large" if you have enough RAM
    print(f"\nModel: {model_name_check} ({models_info[model_name_check]})")

    tokenizer_temp = GPT2Tokenizer.from_pretrained(model_name_check)
    model_temp = GPT2LMHeadModel.from_pretrained(model_name_check)
    model_temp.eval()
    tokenizer_temp.pad_token = tokenizer_temp.eos_token
    model_temp.config.pad_token_id = model_temp.config.eos_token_id

    input_ids = tokenizer_temp.encode(prompt, return_tensors="pt")

    with torch.no_grad():
        output = model_temp.generate(
            input_ids,
            max_length=40,
            do_sample=True,
            temperature=0.7
        )
        generated = tokenizer_temp.decode(output[0], skip_special_tokens=True)
    print(f"Generated: {generated}")

    print(f"Config - Layers: {model_temp.config.n_layer}, "
          f"Hidden size: {model_temp.config.n_embd}, "
          f"Attention heads: {model_temp.config.n_head}")

print("\n" + "=" * 80)

printmd("## Key Insight")
printmd("The core decoder-only architecture remains consistent across all sizes. **Larger models** gain capability by having more parameters (layers and hidden dimensions), not by changing their fundamental structure.")

In [ ]:
# -----------------------------------------------------------------------------
# NOTEBOOK 9: Conditional Generation (Prompt Engineering)
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

printmd("# 9. Conditional Generation - Showing Decoder Flexibility 🎯")
printmd("Decoder-only models can perform tasks traditionally reserved for encoder-decoder models by simply treating the entire input/output as a single sequence to be completed.")
printmd("---")

tasks = [
    ("Translation", "Translate English to French:\nEnglish: Hello, how are you?\nFrench:"),
    ("Question Answering", "Q: What is the capital of France?\nA:"),
    ("Summarization", "Summarize this: Python is a high-level programming language known for its simplicity.\nSummary:"),
    ("Completion", "The three laws of robotics are")]

printmd("Decoder-only models can handle various tasks through prompting:")
print("=" * 80)

for task_name, prompt in tasks:
    print(f"\n{task_name.upper()}:")
    print(f"Prompt: {prompt}")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=input_ids.shape[1] + 30,
            temperature=0.7,
            do_sample=True
        )
        result = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Output: {result}")
    print("-" * 80)

printmd("## Key Insight")
printmd("This demonstrates the **flexibility** of the decoder-only architecture. By structuring the prompt carefully (Prompt Engineering), you can condition the generation to perform specific tasks, despite the lack of a separate encoder.")

In [ ]:
# -----------------------------------------------------------------------------
# NOTEBOOK 10: Understanding Token Embeddings
# -----------------------------------------------------------------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Ensure model and tokenizer are loaded
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

printmd("# 10. Understanding Token Embeddings 📈")
printmd("The model starts by converting discrete tokens into dense vectors (embeddings).")
printmd("---")

# Access the token embedding layer
embedding_layer = model.transformer.wte  # Word Token Embeddings

tokens = ["cat", "dog", "computer", "programming"]
printmd("### Token Embeddings:")

# Compute similarity between embeddings
def cosine_similarity(a, b):
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))

embeddings = {}
for token in tokens:
    token_id = tokenizer.encode(token, add_special_tokens=False)[0]
    embedding = embedding_layer.weight[token_id].detach()
    embeddings[token] = embedding

    print(f"Token: '{token}' (ID: {token_id})")
    print(f"Embedding shape: {embedding.shape}")
    print(f"First 10 dimensions: {embedding[:10].numpy()}\n")

printmd("### Embedding Similarities (Cosine Similarity):")
print(f"cat <-> dog: {cosine_similarity(embeddings['cat'], embeddings['dog']):.4f}")
print(f"cat <-> computer: {cosine_similarity(embeddings['cat'], embeddings['computer']):.4f}")
print(f"dog <-> computer: {cosine_similarity(embeddings['dog'], embeddings['computer']):.4f}")

printmd("## Key Insight")
printmd("The initial embeddings provide the semantic foundation. Notice how the similarity between semantically related words ('cat' and 'dog') is often higher than unrelated words ('cat' and 'computer').")